In [1]:
import pandas as pd
import numpy as np
import json
import os
from os.path import join
from tqdm import tqdm

In [5]:
folder = r"C:\Users\tre_i\LBNL FDD Datasets\LBNL_FDD_Data_Sets_SDAHU_all_3\LBNL_FDD_Dataset_SDAHU"

In [11]:
files = sorted(os.listdir(folder))

In [56]:
files

['AHU_annual.csv',
 'coi_bias_-2_annual.csv',
 'coi_bias_-4_annual.csv',
 'coi_bias_2_annual.csv',
 'coi_bias_4_annual.csv',
 'coi_leakage_010_annual.csv',
 'coi_leakage_025_annual.csv',
 'coi_leakage_040_annual.csv',
 'coi_leakage_050_annual.csv',
 'coi_stuck_010_annual.csv',
 'coi_stuck_025_annual.csv',
 'coi_stuck_050_annual.csv',
 'coi_stuck_075_annual.csv',
 'damper_stuck_010_annual.csv',
 'damper_stuck_025_annual.csv',
 'damper_stuck_075_annual.csv',
 'damper_stuck_100_annual_short.csv',
 'oa_bias_-2_annual.csv',
 'oa_bias_-4_annual.csv',
 'oa_bias_2_annual.csv',
 'oa_bias_4_annual.csv']

In [19]:
files[0]

'AHU_annual.csv'

In [59]:
fault_types_dict = {i: file[:file.find('_annual')] for i, file in enumerate(files)}
fault_types_dict[0] = 'no_fault'

In [114]:
dfs = {}
for i, file in tqdm(enumerate(files)):
    df = pd.read_csv(join(folder, file), index_col='Datetime')
    df.index = pd.to_datetime(df.index)
    df['fault_type'] = i
    df['target'] = i
    df = df.set_index("fault_type", append=True)
    df = df.swaplevel(0, 1).sort_index()
    dfs[file] = df

21it [00:43,  2.09s/it]


In [115]:
for k, df in dfs.items():
    dt = df.index.get_level_values(1)   # 1 = Datetime уровень

    if k == 'damper_stuck_100_annual_short.csv':
        assert dt.min() == pd.Timestamp('2018-04-01 01:00:00')
        assert dt.max() == pd.Timestamp('2018-11-01 00:00:00')
        assert df.shape[0] == 214 * 24 * 60 - 60 + 1
    else:
        assert dt.min() == pd.Timestamp('2018-01-01 01:00:00')
        assert dt.max() == pd.Timestamp('2018-12-31 23:59:00')
        assert df.shape[0] == 365 * 24 * 60 - 60


In [28]:
with open('fault_types_dict.json', 'w') as f:
    json.dump(fault_types_dict, f)

In [116]:
data_full_year = pd.concat([dfs[file] for file in files if file != 'damper_stuck_100_annual_short.csv'])
data_full_year = data_full_year.sort_index()
data_damper_stuck_100 = dfs['damper_stuck_100_annual_short.csv']

In [117]:
cut1 = pd.Timestamp("2018-08-01")
cut2 = pd.Timestamp("2018-10-01")

cut_d1 = pd.Timestamp("2018-08-04")
cut_d2 = pd.Timestamp("2018-09-09")

dt_full = data_full_year.index.get_level_values(1)
dt_damper = data_damper_stuck_100.index.get_level_values(1)

train = pd.concat([
    data_full_year[dt_full < cut1],
    data_damper_stuck_100[dt_damper < cut_d1],
]).sort_index()

val = pd.concat([
    data_full_year[(dt_full >= cut1) & (dt_full < cut2)],
    data_damper_stuck_100[(dt_damper >= cut_d1) & (dt_damper < cut_d2)],
]).sort_index()

test = pd.concat([
    data_full_year[dt_full >= cut2],
    data_damper_stuck_100[dt_damper >= cut_d2],
]).sort_index()

In [112]:
train

CHWC_VLV  CHWC_VLV_DM    MA_TEMP  \
fault_type Datetime                                                    
0          2018-01-01 01:00:00  2.635303e-21          0.0  66.374680   
           2018-01-01 01:01:00  2.479578e-21          0.0  66.374680   
           2018-01-01 01:02:00  2.380361e-21          0.0  66.374680   
           2018-01-01 01:03:00 -1.274259e-21          0.0  66.374626   
           2018-01-01 01:04:00  2.749987e-21          0.0  66.374626   
...                                      ...          ...        ...   
20         2018-07-31 23:55:00 -3.341259e-24          0.0  69.751660   
           2018-07-31 23:56:00  1.648425e-23          0.0  69.751660   
           2018-07-31 23:57:00  1.051530e-23          0.0  69.751660   
           2018-07-31 23:58:00  5.232231e-24          0.0  69.751660   
           2018-07-31 23:59:00 -8.480177e-24          0.0  69.751660   

                                   OA_CFM  OA_DMPR  OA_DMPR_DM    OA_TEMP  \
fault_type Datetime                                                         
0          2018-01-01 01:00:00  357730.44      0.0         0.0  10.355011   
           2018-01-01 01:01:00  357730.44      0.0         0.0  10.040033   
           2018-01-01 01:02:00  357730.44      0.0         0.0  10.055031   
           2018-01-01 01:03:00  357730.44      0.0         0.0  10.070026   
           2018-01-01 01:04:00  357730.44      0.0         0.0  10.085022   
...                                   ...      ...         ...        ...   
20         2018-07-31 23:55:00  357730.44      0.0         0.0  70.475000   
           2018-07-31 23:56:00  357730.44      0.0         0.0  70.448030   
           2018-07-31 23:57:00  357730.44      0.0         0.0  70.421005   
           2018-07-31 23:58:00  357730.44      0.0         0.0  70.394030   
           2018-07-31 23:59:00  357730.44      0.0         0.0  70.367004   

                                  RA_CFM  RA_DMPR  RA_DMPR_DM  ...  SF_CS  \
fault_type Datetime                                            ...          
0          2018-01-01 01:00:00  0.853194      1.0         1.0  ...    0.0   
           2018-01-01 01:01:00  0.853057      1.0         1.0  ...    0.0   
           2018-01-01 01:02:00  0.852918      1.0         1.0  ...    0.0   
           2018-01-01 01:03:00  0.852779      1.0         1.0  ...    0.0   
           2018-01-01 01:04:00  0.852641      1.0         1.0  ...    0.0   
...                                  ...      ...         ...  ...    ...   
20         2018-07-31 23:55:00 -0.006221      1.0         1.0  ...    0.0   
           2018-07-31 23:56:00 -0.006219      1.0         1.0  ...    0.0   
           2018-07-31 23:57:00 -0.006220      1.0         1.0  ...    0.0   
           2018-07-31 23:58:00 -0.006219      1.0         1.0  ...    0.0   
           2018-07-31 23:59:00 -0.006218      1.0         1.0  ...    0.0   

                                SF_SPD  SF_SPD_DM        SF_WAT  SYS_CTL  \
fault_type Datetime                                                        
0          2018-01-01 01:00:00     0.9        0.0 -2.791519e-13      0.0   
           2018-01-01 01:01:00     0.9        0.0 -2.790612e-13      0.0   
           2018-01-01 01:02:00     0.9        0.0 -2.789696e-13      0.0   
           2018-01-01 01:03:00     0.9        0.0 -2.788807e-13      0.0   
           2018-01-01 01:04:00     0.9        0.0 -2.787895e-13      0.0   
...                                ...        ...           ...      ...   
20         2018-07-31 23:55:00     0.9        0.0 -1.486243e-17      0.0   
           2018-07-31 23:56:00     0.9        0.0 -1.486068e-17      0.0   
           2018-07-31 23:57:00     0.9        0.0 -1.485215e-17      0.0   
           2018-07-31 23:58:00     0.9        0.0 -1.484055e-17      0.0   
           2018-07-31 23:59:00     0.9        0.0 -1.483010e-17      0.0   

                                ZONE_TEMP_1  ZONE_TEMP_2  ZONE_TEMP_3  \
fault_type Datetime              

In [131]:
train_df = train.drop('target', axis=1)
train_target = train['target']
val_df = val.drop('target', axis=1)
val_target = val['target']
test_df = test.drop('target', axis=1)
test_target = test['target']

In [119]:
train.shape, val.shape, test.shape

((6284340, 31), (1808640, 31), (2725921, 31))

In [132]:
train_df.to_csv('train_df.csv')
print(len(train_df))
train_target.to_csv('train_target.csv')
print(len(train_target))
val_df.to_csv('val_df.csv')
print(len(val_df))
val_target.to_csv('val_target.csv')
print(len(val_target))
test_df.to_csv('test_df.csv')
print(len(test_df))
test_target.to_csv('test_target.csv')
print(len(test_target))

6284340
6284340
1808640
1808640
2725921
2725921
